In [23]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

from scipy.stats import unitary_group

In [2]:
dev = qml.device('default.qubit', wires=2)

In [635]:
def U_phi(x):
    #print(x, np.shape(x))
    # x3 := (pi - x1)(pi - x2)
    x_0, x_1, x_2 = x[0], x[1], x[2]
    #print(x_0, x_1, x_2)
        
    qml.RZ( x_0 , wires=0)
    qml.RZ( x_1 , wires=1)
    
    qml.CNOT(wires=[0,1])
    qml.RZ(x_2,wires=1)
    qml.CNOT(wires=[0,1])

In [636]:
def layer(W): # 6 weights are specified at each layer
    
    # euler angles
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)

    qml.CNOT(wires=[0, 1])

In [637]:
def featuremap(x):
    for i in range(2):
        qml.Hadamard(wires=0)
        qml.Hadamard(wires=1)
        U_phi(x)

In [638]:
@qml.qnode(dev)
def circuit1(weights, x):

    featuremap(x)

    for W in weights:
        layer(W)

    return qml.expval.PauliZ(wires=0)

@qml.qnode(dev)
def circuit2(weights, x):

    featuremap(x)

    for W in weights:
        layer(W)

    return qml.expval.PauliZ(wires=1)

In [639]:
def variational_classifier(var, x): # x is a keyword argument -> fixed (not trained)
    weights = var[0]
    bias = var[1]

    return circuit1(weights, x) * circuit2(weights, x) + bias

In [640]:
def square_loss(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)

    return loss

In [641]:
def accuracy(labels, predictions):
    
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [642]:
def cost(var, X, Y):

    predictions = [variational_classifier(var, x) for x in X]
    return square_loss(Y, predictions) 

In [562]:
random_U = unitary_group.rvs(4)

print("random unitary: ", random_U)

random unitary:  [[-0.15514148-0.09121604j -0.59385956-0.12811468j -0.244873  +0.64863746j
  -0.21446256+0.26803109j]
 [-0.17124079+0.43648642j -0.09940909-0.04996002j -0.39260273-0.26219456j
   0.53467426+0.50893867j]
 [-0.16637596+0.25524889j  0.63944936+0.36521774j -0.35080085+0.37417065j
  -0.29605956+0.11904363j]
 [-0.75727666+0.28542258j  0.0093654 -0.27599292j  0.15138977+0.10217627j
   0.1112498 -0.47230375j]]


In [643]:
@qml.qnode(dev)
def data_label_1(x):
    #print(u)
    #print("label the following:", x)
    featuremap(x)
    qml.QubitUnitary(random_U, wires=[0,1])
    
    return qml.expval.PauliZ(wires=0)

@qml.qnode(dev)
def data_label_2(x):
    #print("label the following:", x)
    featuremap(x)
    qml.QubitUnitary(random_U, wires=[0,1])
    
    return qml.expval.PauliZ(wires=1)

In [644]:
thresh = 0.3

X = np.array([])
Y = np.array([])
ctr = 0 # num valid data pts
maxval = 0.0
minval = 0.0

np.random.seed(0)

while ctr < 40:
    x = np.random.rand(2) * 2 * np.pi
    x = np.append(x, (np.pi - x[0]) * (np.pi - x[1]))
    y_1 = data_label_1(x)
    y_2 = data_label_2(x)
    #print(y_1, y_2, y_1 * y_2)
    if (np.abs(y_1 * y_2) > maxval):
        maxval = y_1 * y_2
        print("new max separation: ", maxval)
    elif (y_1 * y_2 < minval):
        minval = y_1 * y_2
        print("new min separation: ", minval)
        
    if y_1 * y_2 > thresh:
        Y = np.append(Y, +1)
        X = np.append(X, x)
        ctr += 1
        print("+1")
    elif y_1 * y_2 < -1 * thresh:
        Y = np.append(Y, -1)
        X = np.append(X, x)
        ctr += 1
        print("-1")

new max separation:  -0.02747197821830146
new max separation:  -0.019206078962722577
new max separation:  0.0861293931427618
new max separation:  0.08817139391118359
new max separation:  -0.18226192107016162
new max separation:  -0.008446614417286431
new max separation:  -0.12312895540365273
new max separation:  0.22561678889515877
new min separation:  -0.03253532399106718
new min separation:  -0.16488819080860687
new min separation:  -0.16548098305985282
new max separation:  0.3956445577428955
+1
new min separation:  -0.312615221027314
-1
new max separation:  -0.43039659898492755
-1
new max separation:  0.7980422123333192
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
new min separation:  -0.38875386535672096
-1
+1
+1
+1
+1
+1
+1
+1
+1
new min separation:  -0.5161706397693812
-1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1


In [645]:
X = X.reshape(-1, 3)
print(X)
print(Y)

[[ 5.92974406  4.28400329  3.18521381]
 [ 6.21013598  0.64116646 -7.67266612]
 [ 6.13527576  2.94462234 -0.58966669]
 [ 6.13717092  3.80035648  1.97337861]
 [ 3.28766448  0.59024564 -0.37267992]
 [ 3.68516945  0.12633944 -1.6390217 ]
 [ 5.54010667  4.35130431  2.90151037]
 [ 6.00725065  4.04630976  2.5926098 ]
 [ 3.27376955  0.34141565 -0.3701187 ]
 [ 4.98694975  1.40696031 -3.2010161 ]
 [ 5.86983967  3.85766187  1.95361371]
 [ 2.50209673  1.31848716  1.16586854]
 [ 3.25595111  0.16124361 -0.34082812]
 [ 3.2507888   0.82980839 -0.25243794]
 [ 6.169016    3.00568928 -0.41143703]
 [ 5.69605619  4.86348283  4.39850567]
 [ 2.55877174  1.45917015  0.98055101]
 [ 5.40700249  4.56815383  3.23174572]
 [ 2.33013686  1.23812856  1.54457697]
 [ 5.68027845  4.33555507  3.03109542]
 [ 5.98080373  3.61755124  1.35134689]
 [ 2.05272218  1.46237449  1.82845107]
 [ 0.09805579  2.69420298  1.36164695]
 [ 6.21664498  0.41031843 -8.39881113]
 [ 6.08822878  5.68595815  7.49731928]
 [ 5.27278076  1.49377595

In [646]:
num_data = len(Y)
num_train = int(0.5 * num_data)

print(num_data, num_train)

index = np.random.permutation(range(num_data))
X_train = X[index[:num_train]]
Y_train = Y[index[:num_train]]

X_test = X[index[num_train:]]
Y_test = Y[index[num_train:]]

40 20


In [647]:
num_qubits = 2
num_layers = 6
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)

In [648]:
opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

# train the variational classifier
var = var_init
for it in range(100):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size, ))
    X_train_batch = X_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    var = opt.step(lambda v: cost(v, X_train_batch, Y_train_batch), var)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(var, x)) for f in X_train]
    predictions_test = [np.sign(variational_classifier(var, x)) for f in X_test]

    print(Y_train, predictions_train)
    
    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_test = accuracy(Y_test, predictions_test)

    print("Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
          "".format(it+1, cost(var, X, Y), acc_train, acc_test))

Iter:     1 | Cost: 1.3218162 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     2 | Cost: 1.2852410 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     3 | Cost: 1.2269789 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     4 | Cost: 1.1438986 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     5 | Cost: 1.0357078 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     6 | Cost: 0.9181339 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     7 | Cost: 0.7915565 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     8 | Cost: 0.6629287 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:     9 | Cost: 0.5561416 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:    10 | Cost: 0.4673627 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:    11 | Cost: 0.4020251 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:    12 | Cost: 0.3643526 | Acc train: 0.8500000 | Acc validation: 0.8500000 
Iter:    13 | Co